In [ ]:
#the initial data has some problem from row 1177-1640, we need to fix that first

In [ ]:
!pip install accelerate
!pip install transformers[torch] -U
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForTokenClassification, TrainingArguments, Trainer, TrainerCallback
import torch
from sklearn.metrics import f1_score

data = pd.read_csv('/content/Train_Data_Fixed.tsv', delimiter='\t', encoding='utf-8', error_bad_lines=False)
#data["Tag"].fillna("Obscure", inplace=True)
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.6 MB/s eta 0:00:00


<ipython-input-2-74a5ae871574>:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('/content/Train_Data_Fixed.tsv', delimiter='\t', encoding='utf-8', error_bad_lines=False)


Mounted at /content/drive


In [ ]:
#show the sample data
first_10_rows = data.head(50)
print(first_10_rows)

    RN                                              Title       Token  \
0    1  Supreme Nike SB Dunk High By any Means Red US1...     Supreme   
1    1  Supreme Nike SB Dunk High By any Means Red US1...        Nike   
2    1  Supreme Nike SB Dunk High By any Means Red US1...          SB   
3    1  Supreme Nike SB Dunk High By any Means Red US1...        Dunk   
4    1  Supreme Nike SB Dunk High By any Means Red US1...        High   
5    1  Supreme Nike SB Dunk High By any Means Red US1...          By   
6    1  Supreme Nike SB Dunk High By any Means Red US1...         any   
7    1  Supreme Nike SB Dunk High By any Means Red US1...       Means   
8    1  Supreme Nike SB Dunk High By any Means Red US1...         Red   
9    1  Supreme Nike SB Dunk High By any Means Red US1...        US10   
10   1  Supreme Nike SB Dunk High By any Means Red US1...        EU44   
11   1  Supreme Nike SB Dunk High By any Means Red US1...     Supreme   
12   1  Supreme Nike SB Dunk High By any Means Red 

In [ ]:
print(data.tail(10))

         RN                                              Title        Token  \
55173  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...       Herren   
55174  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...     Trekking   
55175  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...       Schuhe   
55176  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...      Outdoor   
55177  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...      Sneaker   
55178  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...  Sportschuhe   
55179  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...  Wanderschuh   
55180  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...          Big   
55181  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...         Size   
55182  5000  Herren Trekking Schuhe Outdoor Sneaker Sportsc...          U37   

              Tag  Extra  
55173   Abteilung    NaN  
55174   Aktivität    NaN  
55175  Produktart    NaN  
55176      Anlass    N

In [ ]:
print(data.shape)

(55183, 5)


In [ ]:
def turn_all_the_tag_to_string(df):
  for index,row in df.iterrows():
    if pd.isna(row['Tag']):
            print('Original value is NaN, converting to string "NaN".')
            df.at[index, 'Tag'] = 'NaN'
  return df

In [ ]:
data = turn_all_the_tag_to_string(data)

流式输出内容被截断，只能显示最后 5000 行内容。
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Original value is NaN, converting to string "NaN".
Orig

In [ ]:
def print_type_tag(df):
  for index,row in df.iterrows():
    print(type(row['Tag']))

In [ ]:
print_type_tag(data)

流式输出内容被截断，只能显示最后 5000 行内容。
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 

In [ ]:

def update_tags(df):
    # Initialize a variable to keep track of the current group tag
    current_tag = None
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        # If the tag is not 'NaN', update the current tag and continue
        current_tag= row['Tag']
        '''if row['Tag'] != 'NaN':
            current_tag = row['Tag']
            continue'''
        #print(current_tag)
        #print(type(current_tag))
        # If the current tag is not None and the previous tag is not 'NaN', it means it's part of a group
        if current_tag == 'NaN' and df.loc[index - 1, 'Tag'] != 'NaN':
            # Update the tag to the current tag with the '-in' suffix
            if df.loc[index - 1, 'Tag'][-3:]!='-in':
              df.at[index, 'Tag'] = df.loc[index - 1, 'Tag'] + '-in'
            else:
              df.at[index, 'Tag'] = df.loc[index - 1, 'Tag']

    return df

In [ ]:
data=update_tags(data)
first_10_rows=data.head(10)
print(first_10_rows)

   RN                                              Title    Token  \
0   1  Supreme Nike SB Dunk High By any Means Red US1...  Supreme   
1   1  Supreme Nike SB Dunk High By any Means Red US1...     Nike   
2   1  Supreme Nike SB Dunk High By any Means Red US1...       SB   
3   1  Supreme Nike SB Dunk High By any Means Red US1...     Dunk   
4   1  Supreme Nike SB Dunk High By any Means Red US1...     High   
5   1  Supreme Nike SB Dunk High By any Means Red US1...       By   
6   1  Supreme Nike SB Dunk High By any Means Red US1...      any   
7   1  Supreme Nike SB Dunk High By any Means Red US1...    Means   
8   1  Supreme Nike SB Dunk High By any Means Red US1...      Red   
9   1  Supreme Nike SB Dunk High By any Means Red US1...     US10   

               Tag  Extra  
0           Modell    NaN  
1            Marke    NaN  
2     Produktlinie    NaN  
3  Produktlinie-in    NaN  
4  Schuhschaft-Typ    NaN  
5           Modell    NaN  
6        Modell-in    NaN  
7        Modell-

In [ ]:
print(data.shape)

(55183, 5)


In [ ]:
#import the pretrained bert model
MAX_LEN = 75
BATCH_SIZE = 32
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-german-cased')

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

In [ ]:
words = list(set(data["Token"].values))
words.append("ENDPAD")
tags = sorted(list(set(data["Tag"].values)))
# tags.append("No Tag")x
print(tags)
print(len(tags))
#the tags which will not appear in the prediction file is no tag, no tag-in and obscure

['Abteilung', 'Abteilung-in', 'Aktivität', 'Aktivität-in', 'Akzente', 'Akzente-in', 'Anlass', 'Anlass-in', 'Besonderheiten', 'Besonderheiten-in', 'Charakter', 'Charakter Familie', 'Charakter Familie-in', 'Charakter-in', 'Dämpfungsgrad', 'Dämpfungsgrad-in', 'EU-Schuhgröße', 'EU-Schuhgröße-in', 'Erscheinungsjahr', 'Erscheinungsjahr-in', 'Farbe', 'Farbe-in', 'Futtermaterial', 'Futtermaterial-in', 'Gewebeart', 'Gewebeart-in', 'Herstellernummer', 'Herstellernummer-in', 'Herstellungsland und -region', 'Herstellungsland und -region-in', 'Innensohlenmaterial', 'Innensohlenmaterial-in', 'Jahreszeit', 'Jahreszeit-in', 'Laufsohlenmaterial', 'Marke', 'Marke-in', 'Maßeinheit', 'Maßeinheit-in', 'Modell', 'Modell-in', 'Muster', 'Muster-in', 'No Tag', 'No Tag-in', 'Obermaterial', 'Obermaterial-in', 'Obscure', 'Produktart', 'Produktart-in', 'Produktlinie', 'Produktlinie-in', 'Schuhschaft-Typ', 'Schuhschaft-Typ-in', 'Schuhweite', 'Stil', 'Stil-in', 'Stollentyp', 'Thema', 'Thema-in', 'UK-Schuhgröße', 'UK

In [ ]:
#the class to process the dataset, apply masks to the dataset
class NerDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __getitem__(self, idx):

        #print(f"Length of input_ids: {len(self.input_ids)}")
        #print(f"Length of attention_masks: {len(self.attention_masks)}")
        #print(f"Length of labels: {len(self.labels)}")

        #print(f"index {idx}")

        '''if idx >= len(self.input_ids):
            raise IndexError("Index out of bounds for input_ids")
        if idx >= len(self.attention_masks):
            raise IndexError("Index out of bounds for attention_masks")
        if idx >= len(self.labels):
            raise IndexError("Index out of bounds for labels")'''

        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_masks[idx],
            "labels": self.labels[idx]
        }

    def __len__(self):
        return len(self.input_ids)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
idx2tag = {i: t for t, i in tag2idx.items()}

agg_func = lambda s: [(w, t) for w, t in zip(s["Token"].values.tolist(), s["Tag"].values.tolist())]
grouped = data.groupby("RN").apply(agg_func)
sentences = [s for s in grouped]

tokenized_texts = [tokenizer.tokenize(" ".join([str(s[0]) for s in sent])) for sent in sentences]
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0, truncating="post", padding="post")


In [ ]:
#deal with the problem of the subwords are genearted through the tokenizer
#we give the subwords genearted from the same word the same token corresponding to the token of the word
def align_labels_with_subwords2(tokenized_texts, sentences):
    aligned_labels = []

    for tokens, sentence in zip(tokenized_texts, sentences):
        labels = []
        word_index = 0
        is_prev_token_hyphenated = False

        print('the length of the sentence is ',len(sentence))
        print(tokens)
        print(sentence)

        for i, token in enumerate(tokens):
            # Check if the current token is a period or hyphen
            print('the index of the word is',word_index)
            print('the token is ',token)
            if token in ['.', '-'] and i > 0:
                # Check if the previous full token includes a period or hyphen
                prev_full_token = sentence[word_index][0] if word_index < len(sentence) else ''
                if '.' in prev_full_token or '-' in prev_full_token:
                    # Do not increment word index and assign the same label as the previous token
                    labels.append(labels[-1])
                    continue

            # Check if the token is the start of a new word
            if not token.startswith('##') and token not in ['.', '-']:
                # Update the word label only if it's a new word
                if word_index < len(sentence):
                    word_label = sentence[word_index][1]
                    labels.append(word_label)
                    word_index += 1
                else:
                    # If there are no more words in the sentence, use the last word's label
                    labels.append(labels[-1])
            else:
                # For subword tokens or '.', '-' tokens that are part of the previous word, use the previous label
                labels.append(labels[-1])

            # Check if the current token is a hyphenated part
            print(sentence[word_index-1][0])
            is_prev_token_hyphenated = token.startswith('##') and '-' in sentence[word_index - 1][0] if word_index > 0 else False

        aligned_labels.append(labels)

    return aligned_labels


In [ ]:
#this is the wrong version of labels and it can't match with the tokenized tokens
#labels = [[s[1] for s in sent] for sent in sentences]

#get the aligned labels which can match well with the tokenized tokens
aligned_labels=align_labels_with_subwords2(tokenized_texts,sentences)

tags_padded = pad_sequences([[tag2idx[l] for l in lab] for lab in aligned_labels],
                            maxlen=MAX_LEN, value=tag2idx["No Tag"], padding="post",
                            dtype="long", truncating="post")

tags_flat = np.array(tags_padded).flatten()
print(tag2idx["No Tag"])
print("Number of unique labels in tag2idx:", len(tag2idx))
print("Maximum label index in padded tags:", tags_flat.max())

流式输出内容被截断，只能显示最后 5000 行内容。
Sneaker
the index of the word is 7
the token is  Sports
Sportschuhe
the index of the word is 8
the token is  ##ch
Sportschuhe
the index of the word is 8
the token is  ##uh
Sportschuhe
the index of the word is 8
the token is  ##e
Sportschuhe
the index of the word is 8
the token is  Schuhe
Schuhe
the index of the word is 9
the token is  Grau
Grau
the index of the word is 10
the token is  Gelb
Gelb
the index of the word is 11
the token is  Größe
Größe
the index of the word is 12
the token is  45
45,5
the index of the word is 13
the token is  ,
45,5
the index of the word is 13
the token is  5
45,5
the length of the sentence is  12
['Joe', 'Ni', '##mb', '##le', 'Herren', 'Bar', '##fus', '##schuhe', 'Bar', '##ef', '##oo', '##t', 'Vi', '##bra', '##ms', '##oh', '##le', 'Sn', '##ea', '##ker', '-', 'Gr', '.', '47', '-', 'schwarz']
[('Joe', 'Marke'), ('Nimble', 'Marke-in'), ('Herren', 'Abteilung'), ('Barfusschuhe', 'Produktart'), ('Barefoot', 'Dämpfungsgrad'), ('Vibrams

In [ ]:
from sklearn.metrics import f1_score
from transformers import TrainerCallback, IntervalStrategy
from typing import Dict, Union
import pandas as pd

# Define the compute metrics function
def compute_metrics2(p):
    true_labels = p.label_ids
    pred_labels = p.predictions.argmax(-1)

    # Flatten the labels and predictions
    true_labels = true_labels.reshape(-1)
    pred_labels = pred_labels.reshape(-1)

    #write the true labels and pred labels to a csv file to see what exactly it is

    NON_LABEL_ID = 43
    mask = true_labels != NON_LABEL_ID
    filtered_true_labels = true_labels[mask]
    filtered_pred_labels = pred_labels[mask]

    # Calculate F1 for non 'NO_LABEL' labels
    weighted_f1 = f1_score(filtered_true_labels, filtered_pred_labels, average='weighted', labels=np.unique(pred_labels))
    return {'weighted_f1': weighted_f1}

In [ ]:
class F1ScoreCallback(TrainerCallback):
    """自定义 Callback，用于在日志中打印加权 F1 分数"""

    def on_log(self, args, state, control, logs=None, **kwargs):
        # logs 字典中包含了所有的指标，包括加权 F1 分数
        if 'eval_weighted_f1' in logs:
            print(f"Step {state.global_step}: Weighted F1 = {logs['eval_weighted_f1']:.4f}")

In [ ]:
#add the attention mask to the model
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
#define the models for training
from transformers import BertTokenizer, BertForTokenClassification
from torch.utils.data import DataLoader, Subset, RandomSampler
import torch
import numpy as np

model = BertForTokenClassification.from_pretrained(
    "dbmdz/bert-base-german-cased",
    output_attentions=False,
    output_hidden_states=False,
    num_labels=68
)
model.to('cuda')

model = BertForTokenClassification.from_pretrained("dbmdz/bert-base-german-cased", num_labels=68)

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#create the evaluation dataset
train_indices, val_indices, train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    range(len(sentences)),
    input_ids,
    tags_padded,
    test_size=0.2,
    random_state=56
)

#also split the attention mask using the same method
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.2)

tr_inputs = torch.tensor(train_sentences)
val_inputs = torch.tensor(val_sentences)
tr_tags = torch.tensor(train_labels)
val_tags = torch.tensor(val_labels)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

eval_dataset = NerDataset(val_inputs, val_masks, val_tags)

In [ ]:
#create the overall dataset
import torch

inputs=torch.tensor(input_ids)
labels=torch.tensor(tags_padded)
masks=torch.tensor(attention_masks)

dataset= NerDataset(inputs,masks,labels)

In [ ]:
num_subsets = 5

subsets = [Subset(dataset, np.random.choice(len(dataset), size=len(dataset)//num_subsets, replace=True)) for _ in range(num_subsets)]

models = []

num_subset=0
for subset in subsets:
  num_subset+=1
  subset_model = BertForTokenClassification.from_pretrained("dbmdz/bert-base-german-cased", num_labels=68)

  training_args = TrainingArguments(
    output_dir=f'/content/drive/My Drive/Model_subset_{num_subset}/',
    num_train_epochs=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=300,
    weight_decay=0.01,
    logging_dir='./logs',
    learning_rate=3e-5,
    gradient_accumulation_steps=2,
    logging_steps=100,
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="weighted_f1",  # Change this to use weighted F1 as the best metric
    )

  trainer = Trainer(
    model=subset_model,
    args=training_args,
    train_dataset=subsets[num_subsets-1],
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics2, # Add compute metrics function here
     callbacks=[F1ScoreCallback()]
    )

  trainer.train()

  models.append(subset_model)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Weighted F1
500,0.109800,0.664252,0.699976
1000,0.016100,0.693607,0.731132
1500,0.004900,0.699669,0.736198
2000,0.002700,0.731865,0.734748
2500,0.001500,0.745592,0.736832
3000,0.001200,0.767616,0.738460


Step 500: Weighted F1 = 0.7000
Step 1000: Weighted F1 = 0.7311
Step 1500: Weighted F1 = 0.7362
Step 2000: Weighted F1 = 0.7347
Step 2500: Weighted F1 = 0.7368
Step 3000: Weighted F1 = 0.7385


KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
from transformers import BertTokenizer, BertForTokenClassification

model_fold_1_path = "/content/drive/My Drive/Model_Fold_1/checkpoint-4000"  # Path to the shared folder
model_fold_1_4000 = BertForTokenClassification.from_pretrained(model_fold_1_path)

model_fold_2_path = "/content/drive/My Drive/Model_Fold_2/checkpoint-4000"
model_fold_2_4000 = BertForTokenClassification.from_pretrained(model_fold_2_path)

model_fold_3_path = "/content/drive/My Drive/Model_Fold_3/checkpoint-4000"
model_fold_3_4000 = BertForTokenClassification.from_pretrained(model_fold_3_path)

model_fold_4_path = "/content/drive/My Drive/Model_Fold_4/checkpoint-4000"
model_fold_4_4000 = BertForTokenClassification.from_pretrained(model_fold_4_path)

model_fold_5_path = "/content/drive/My Drive/Model_Fold_5/checkpoint-4000"
model_fold_5_4000 = BertForTokenClassification.from_pretrained(model_fold_5_path)

model_fold_6_path = "/content/drive/MyDrive/Model_Fold_21/checkpoint-5000"
model_fold_6_5000 = BertForTokenClassification.from_pretrained(model_fold_6_path)

model_fold_7_path = "/content/drive/MyDrive/Model_Fold_21/checkpoint-6500"
model_fold_7_6500 = BertForTokenClassification.from_pretrained(model_fold_7_path)

model_fold_8_path = "/content/drive/MyDrive/Model_Fold_21/checkpoint-7500"
model_fold_8_7500 = BertForTokenClassification.from_pretrained(model_fold_8_path)

model_fold_9_path = "/content/drive/MyDrive/Model_Fold_22/checkpoint-3000"
model_fold_9_3000 = BertForTokenClassification.from_pretrained(model_fold_9_path)

model_fold_10_path = "/content/drive/MyDrive/Model_Fold_22/checkpoint-4500"
model_fold_10_4500 = BertForTokenClassification.from_pretrained(model_fold_10_path)

model_name = "dbmdz/bert-base-german-cased"

tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
models=[model_fold_1_4000,model_fold_2_4000,model_fold_3_4000,model_fold_4_4000,model_fold_5_4000,model_fold_6_5000,model_fold_7_6500,model_fold_8_7500,model_fold_9_3000,model_fold_10_4500]

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
class EnsembleModel(nn.Module):
    def __init__(self, model_list):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList(model_list)
        for model in self.models:
            model.eval()

    def forward(self, input_ids, attention_mask=None):
        avg_softmax = None
        for model in self.models:
            output = model(input_ids, attention_mask=attention_mask)
            logits = output.logits
            softmax = F.softmax(logits, dim=-1)
            if avg_softmax is None:
                avg_softmax = softmax
            else:
                avg_softmax += softmax
        avg_softmax /= len(self.models)
        return torch.argmax(avg_softmax, dim=-1)

In [ ]:
esemble_model=EnsembleModel(models)

In [ ]:
from google.colab import drive
from transformers import BertTokenizer, BertForTokenClassification
#drive.mount('/content/drive')
# Load the model from the shared folder
model_path = '/content/drive/MyDrive/Model_Fold_4/checkpoint-4000'  # Path to the shared folder

model_name = "dbmdz/bert-base-german-cased"

model_4000 = BertForTokenClassification.from_pretrained(model_path)

#checkpoint = torch.load()
#model_5000.load_state_dict(checkpoint)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
def decode_tokens(tokens):
  return tokenizer.convert_tokens_to_string(tokens)

In [ ]:
#define the trainer for the model which is fineuned for 1500 epochs
trainer_2 = Trainer(
    model=esemble_model,
    # Add compute metrics function here
)

In [ ]:
predictions_eval,_,_=trainer_2.predict(eval_dataset) #there might be something wrong with this predict method,
#why on the evaluation set the weighted f1 score is so high but when we print out the result, the prediction accuracy is so bad?
#could it be because the no tags are contained when calculating the weighted f1 score in the validation steps during training? for the no tag has large weight
#also, on the test set, the f1 score is so low

#predicted_labels = np.argmax(predictions_eval, axis=2)

NameError: ignored

In [ ]:
print(len(predictions_eval))

1000


In [ ]:
print(idx2tag[35])

Marke


In [ ]:
#convert the prediction to a list?
from copy import deepcopy
num_rows = len(predicted_labels)
num_cols = len(predicted_labels[0])
predicted_label_name = [[0 for _ in range(num_cols)] for _ in range(num_rows)]
#print(predicted_label_name)
#print(len(predicted_label_name))
#print(len(predicted_label_name[0]))
for i in range(len(predicted_labels)):
  for j in range(len(predicted_labels[0])):
    #print(predicted_label_name[i][j])
    predicted_label_name[i][j]=idx2tag[predicted_labels[i][j]]
print(predicted_label_name)

[['Marke', 'Marke', 'Marke', 'Produktlinie', 'Produktlinie', 'Produktlinie-in', 'Produktlinie-in', 'Modell', 'Modell', 'Abteilung', 'Produktart', 'Stil', 'Stil', 'Stil', 'Produktart', 'Produktart', 'Herstellernummer', 'Herstellernummer', 'Herstellernummer', 'No Tag', 'Farbe', 'Farbe', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag', 'No Tag'], ['Marke', 'Marke', 'Produktlinie', 'Produktlinie-in', 'Modell', 'Modell-in', 'Modell-in', 'Aktivität', 'Aktivität', 'Stil', 'Stil', 'Stil', 'Produktart', 'Produktart', 'Produktart-in'

In [ ]:
from collections import Counter

def most_common_number(lst):
    counts = Counter(lst)
    max_count = max(counts.values())  # 找到最大的出现次数
    most_common = [num for num, count in counts.items() if count == max_count]  # 找到出现次数最多的数字列表
    most_common.sort(key=lambda x: lst.index(x))  # 根据数字在原始列表中的索引排序
    return most_common[0]  # 返回出现次数最多且索引最靠前的数字

# 示例列表
my_list = [1, 2, 3, 2, 2, 3, 4, 4, 4, 5]

result = most_common_number(my_list)
print(result)  # 输出出现次数最多且索引最靠前的数字

2


In [ ]:
def get_formatted_prediction(records,tokens,predictions,Titles):
  #create the list of Non tags
  counter=0

  lst_non_tag_words=['.','#','-','(',')','!!','[',']','/','...','*',',','****','--','(/','....']

  formatted_predictions_eval = []
  predictions_for_words=[]
  title_sentence = [sentence.split() for sentence in Titles]
  for records_eval, tokens_eval, preds_eval, title in zip(records, tokens, predictions, title_sentence):
      discrminant=False
      #word_tokens_eval=decode_tokens(tokens_eval).split()
      #print('the record number is',records_eval)
      #print('token length', len(tokens_eval))
      #print(tokens_eval)
      #print(preds_eval)
      #print(word_tokens_eval)
      #print(title)

      preds_word_labels=[]
      index=0
      index_title=0
      while index < len(tokens_eval) and index_title<len(title):
        preds_one_word=[preds_eval[index]]
        tokens_eval_pre_word=tokens_eval[index]
        if index+1 < len(tokens_eval) and tokens_eval[index]== '[UNK]':
          if tokens_eval[index+1] not in title[index_title]:
            index += 1
            index_title +=1
            result = most_common_number(preds_one_word)
            preds_word_labels.append(result)
            continue

        index += 1
        #print(index)


        while index < len(tokens_eval) and (tokens_eval[index].startswith("##") or tokens_eval_pre_word != title[index_title]):
          preds_one_word.append(preds_eval[index])
          if '[UNK]' in tokens_eval_pre_word :
            if tokens_eval[index].startswith("##"):
              tokens_eval_pre_word += tokens_eval[index].replace('##', '')
              if index+1 < len(tokens_eval) and tokens_eval[index+1] not in title[index_title]:
                index += 1
                break
            else:
              tokens_eval_pre_word += tokens_eval[index]
              if index+1 < len(tokens_eval) and not tokens_eval[index+1].startswith("##"):
                if tokens_eval[index+1] not in title[index_title]:
                  index += 1
                  break
          if tokens_eval[index].startswith("##"):
            tokens_eval_pre_word += tokens_eval[index].replace('##', '')
          elif tokens_eval[index]== '[UNK]':
            if index+1 < len(tokens_eval):
              if tokens_eval[index+1] in title[index_title]:
                tokens_eval_pre_word += tokens_eval[index]
              else:
                tokens_eval_pre_word += tokens_eval[index]
                index += 1
                break
          else:
            tokens_eval_pre_word += tokens_eval[index]
          index += 1

        '''if index<len(tokens_eval):
          if tokens_eval[index]=='[UNK]':
            #print('hi')
            #print(records_eval)
            #print('the record number is',records_eval)
            #print('token length', len(tokens_eval))
            #print(tokens_eval)
            #print(preds_eval)
            #print(tokens_eval_pre_word)
            #print(title)
            #print("preds_word_labels",preds_word_labels)
            #tokens_eval_pre_word+='what a fuck'
            index+=1
            index_title+=1
            preds_word_labels.append(43)
            discrminant=True'''

        #print(tokens_eval_pre_word)
        index_title +=1
        result = most_common_number(preds_one_word)
        preds_word_labels.append(result)

      #print("preds_word_labels",preds_word_labels)

      predictions_for_words.append(preds_word_labels)

      prev_label_id = None
      combined_token = ""
      if len(title)!=len(preds_word_labels) or discrminant==True:
        print('the record number is',records_eval)
        print('token length', len(tokens_eval))
        print(tokens_eval)
        print(preds_eval)
        print(tokens_eval_pre_word)
        print(title)
        print("preds_word_labels",preds_word_labels)
        counter+=1
        print(counter)
        #print(title)
      if len(title)!=len(preds_word_labels):
          continue
      for word, label_id in zip(title, preds_word_labels):
          #print(word,label_id)
          #print('this is the combined token',combined_token)
          if idx2tag[label_id] in ["No Tag", "Obscure","No Tag-in"] or word in lst_non_tag_words:
              #print('hi')
              #prev_label_id=label_id
              #print(prev_label_id)
              continue
          #print(idx2tag[label_id][:-3])
          if idx2tag[label_id][-3:]=='-in':
              if prev_label_id==None:
                prev_label_id=tag2idx[idx2tag[label_id][:-3]]
                combined_token=word
                continue
              elif idx2tag[label_id][:-3]==idx2tag[prev_label_id]:
                combined_token += " " + word
                new_id=idx2tag[label_id]
                new_id=new_id[:-3]
                label_id=tag2idx[new_id]
              else:
                #print('hi',word)
                if combined_token:
                  formatted_predictions_eval.append([records_eval, idx2tag[prev_label_id], combined_token])
                prev_label_id=tag2idx[idx2tag[label_id][:-3]]
                combined_token = word
                continue

          else:
              if combined_token:
                  formatted_predictions_eval.append([records_eval, idx2tag[prev_label_id], combined_token])
                  combined_token = ""
              combined_token = word



          prev_label_id = label_id

      if combined_token:
        formatted_predictions_eval.append([records_eval, idx2tag[prev_label_id], combined_token])

  return formatted_predictions_eval,predictions_for_words

In [ ]:
formatted_predictions_eval,predictions_for_words_eval=get_formatted_prediction(unique_lst_records_eval, tokens_eval, predicted_labels,third_col_list)

In [ ]:
print(predictions_for_words_eval)

[[35, 50, 51, 39, 0, 48, 55, 48, 26, 43, 20, 43], [35, 50, 51, 39, 40, 2, 55, 48, 49, 26, 27], [35, 0, 64, 39, 40, 40, 20, 21, 21, 43, 43, 43], [35, 50, 39, 40, 40, 39, 43, 35, 43, 39, 39, 43, 43], [35, 50, 39, 40, 0, 48, 55, 48, 26, 20, 43], [0, 1, 35, 39, 50, 45, 43, 50, 43, 16, 26, 26], [35, 43, 11, 50, 39, 50, 43, 10, 13, 13, 13, 43, 62, 63, 16, 17, 43, 43, 43, 43], [48, 39, 40, 40, 35, 48, 43, 8, 8], [35, 50, 26, 39, 48, 39, 55, 48, 43, 43, 43, 16, 17, 62, 63, 43, 43], [35, 50, 39, 40, 0, 48, 43, 55, 43, 2, 43, 2, 43, 26, 27, 43], [0, 48, 35, 39, 2, 26], [35, 50, 51, 39, 40, 40, 16, 62, 60, 39, 40, 43, 43, 43, 43], [35, 36, 0, 55, 45, 46, 46, 46, 20, 43, 43, 43, 43, 43], [35, 36, 39, 40, 0, 20, 41, 20, 21, 48, 55, 20, 21], [35, 0, 47, 47, 43, 20, 20, 21, 21, 21, 26, 26], [35, 39, 0, 55, 48, 43, 39, 20, 43, 43, 43], [35, 36, 43, 43, 43, 0, 20, 45, 43, 45, 55, 43], [35, 50, 39, 0, 1, 55, 48, 49, 20, 26, 43, 43, 43, 16, 43], [50, 51, 51, 58, 20, 21, 43, 43, 43, 16, 62], [35, 47, 46, 

In [ ]:
#get the ground truth labels for validation set
sequences = []

print(data.keys())

print(val_indices)

# Iterate through each index in val_indices
for index in val_indices:
    # Filter the dataframe to get rows with the current index
    sentence_df = data[data['RN'] == index+1]

    # Get the tags, convert them to indices using tag2idx, and append to sequences
    tag_sequence = [tag2idx.get(tag, -1) for tag in sentence_df['Tag'].tolist()]
    sequences.append(tag_sequence)

print(sequences)

Index(['RN', 'Title', 'Token', 'Tag', 'Extra'], dtype='object')
[1501, 2586, 2653, 1055, 705, 106, 589, 2468, 2413, 1600, 2464, 228, 915, 794, 3021, 3543, 1073, 3351, 1744, 1084, 926, 3049, 1117, 642, 4767, 501, 4066, 333, 4684, 486, 1962, 393, 4842, 4866, 1755, 2515, 3585, 4315, 4966, 2099, 3599, 4121, 29, 65, 838, 3906, 3773, 4635, 3161, 2659, 4615, 4628, 2451, 2846, 1144, 3078, 1103, 168, 1670, 2570, 2377, 4395, 4257, 3862, 23, 2633, 3340, 2215, 3682, 4724, 1907, 84, 227, 296, 1001, 2138, 711, 2801, 2527, 3752, 3321, 3181, 4183, 1615, 3024, 1413, 763, 655, 4797, 3849, 4153, 468, 157, 1295, 497, 4740, 2940, 3456, 373, 79, 1553, 1669, 2131, 4439, 465, 248, 931, 1335, 4524, 1338, 1406, 1090, 4209, 742, 2323, 3601, 2123, 2094, 240, 2351, 1370, 1057, 4816, 199, 2280, 4749, 2220, 4771, 2819, 3150, 4256, 3107, 1973, 1375, 3968, 4062, 4445, 3557, 964, 179, 3698, 3821, 4878, 3899, 4487, 3238, 2974, 3580, 2887, 2575, 1419, 3907, 4673, 4902, 1860, 8, 2894, 2847, 3664, 144, 2104, 3143, 841, 112

In [ ]:
from sklearn.metrics import classification_report

print(len(predictions_for_words_eval))
print(len(sequences))

flattened_predictions = []
flattened_ground_truth = []
count=0
for pred, true in zip(predictions_for_words_eval, sequences):
    # Truncate predictions to the length of the ground truth for each sentence
    #print('length of prediction',len(pred))
    #print('length of true prediction',len(true))
    if len(flattened_predictions)!=len(flattened_ground_truth):
      print(pred)
      print(true)
      print(len(flattened_predictions))
      print(len(flattened_ground_truth))
      print(flattened_predictions)
      print(flattened_ground_truth)
      print(count)
      break
    if len(pred)>len(true):
      truncated_pred = pred[:len(true)]

    #print('length of truncated prediction',len(truncated_pred))
      flattened_predictions.extend(truncated_pred)
      flattened_ground_truth.extend(true)
    elif len(pred)<len(true):
      truncated_ture=true[:len(pred)]

      flattened_predictions.extend(pred)
      flattened_ground_truth.extend(truncated_ture)

    else:
      flattened_predictions.extend(pred)
      flattened_ground_truth.extend(true)
    count+=1

# Compute weighted F1 score and classification report
report = classification_report(flattened_ground_truth, flattened_predictions, output_dict=True )

print(report)

1000
1000
{'0': {'precision': 0.8928046989720999, 'recall': 0.8798842257597684, 'f1-score': 0.8862973760932944, 'support': 691}, '1': {'precision': 0.8518518518518519, 'recall': 0.8846153846153846, 'f1-score': 0.8679245283018868, 'support': 52}, '2': {'precision': 0.8083333333333333, 'recall': 0.8220338983050848, 'f1-score': 0.8151260504201682, 'support': 118}, '3': {'precision': 0.7647058823529411, 'recall': 0.6842105263157895, 'f1-score': 0.7222222222222222, 'support': 19}, '4': {'precision': 0.8947368421052632, 'recall': 0.85, 'f1-score': 0.8717948717948718, 'support': 20}, '5': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4}, '6': {'precision': 0.8938053097345132, 'recall': 0.8632478632478633, 'f1-score': 0.8782608695652174, 'support': 117}, '7': {'precision': 1.0, 'recall': 0.7777777777777778, 'f1-score': 0.8750000000000001, 'support': 9}, '8': {'precision': 0.7567567567567568, 'recall': 0.8115942028985508, 'f1-score': 0.7832167832167832, 'support': 69}, '9': {'pr

In [ ]:
print(flattened_predictions)
print(flattened_ground_truth)

[35, 50, 51, 39, 0, 48, 55, 48, 26, 43, 20, 43, 35, 50, 51, 39, 40, 2, 55, 48, 49, 26, 27, 35, 0, 64, 39, 40, 40, 20, 21, 21, 43, 43, 43, 35, 50, 39, 40, 40, 39, 43, 35, 43, 39, 39, 43, 43, 35, 50, 39, 40, 0, 48, 55, 48, 26, 20, 43, 0, 1, 35, 39, 50, 45, 43, 50, 43, 16, 26, 26, 35, 43, 11, 50, 39, 50, 43, 10, 13, 13, 13, 43, 62, 63, 16, 17, 43, 43, 43, 43, 48, 39, 40, 40, 35, 48, 43, 8, 8, 35, 50, 26, 39, 48, 39, 55, 48, 43, 43, 43, 16, 17, 62, 63, 43, 43, 35, 50, 39, 40, 0, 48, 43, 55, 43, 2, 43, 2, 43, 26, 27, 43, 0, 48, 35, 39, 2, 26, 35, 50, 51, 39, 40, 40, 16, 62, 60, 39, 40, 43, 43, 43, 43, 35, 36, 0, 55, 45, 46, 46, 46, 20, 43, 43, 43, 43, 43, 35, 36, 39, 40, 0, 20, 41, 20, 21, 48, 55, 20, 21, 35, 0, 47, 47, 43, 20, 20, 21, 21, 21, 26, 26, 35, 39, 0, 55, 48, 43, 39, 20, 43, 43, 43, 35, 36, 43, 43, 43, 0, 20, 45, 43, 45, 55, 43, 35, 50, 39, 0, 1, 55, 48, 49, 20, 26, 43, 43, 43, 16, 43, 50, 51, 51, 58, 20, 21, 43, 43, 43, 16, 62, 35, 47, 46, 8, 8, 35, 50, 51, 39, 40, 52, 39, 40, 4

In [ ]:
df_formatted_predictions_eval = pd.DataFrame(formatted_predictions_eval)
#save the prediction for evaluation set as csv file
csv_file_path = "/content/drive/MyDrive/Validation&Training/prediction_eval_4000_11.27.csv"
df_formatted_predictions_eval.to_csv(csv_file_path, index=False)

In [ ]:
#We can test our prediction on the first 5000 rows of the listing titles
listing_data = pd.read_csv('/content/Listing_Titles.tsv', delimiter='\t', encoding='utf-8', error_bad_lines=False)
train_validation_data_tokens = listing_data.iloc[0:5000]["Title"].tolist()

tokenizer_texts_train_validation=[tokenizer.tokenize(t) for t in train_validation_data_tokens]
input_ids_train_validation = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenizer_texts_train_validation],
                               maxlen=MAX_LEN, dtype="long", value=0.0,
                               truncating="post", padding="post")

attention_masks_test = [[float(i != 0.0) for i in ii] for ii in input_ids_train_validation]

input_ids_train_validation = torch.tensor(input_ids_train_validation)
attention_masks_train_validation = torch.tensor(attention_masks_test)

dummy_label_train_validation = torch.zeros_like(input_ids_train_validation)

<ipython-input-31-334543c6a648>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  listing_data = pd.read_csv('/content/Listing_Titles.tsv', delimiter='\t', encoding='utf-8', error_bad_lines=False)


In [ ]:
print(input_ids_train_validation)
print(attention_masks_train_validation)
print(dummy_label_train_validation)

tensor([[26917,  7909, 30881,  ...,     0,     0,     0],
        [ 1942, 30819,  9912,  ...,     0,     0,     0],
        [  101,   146,  5085,  ...,     0,     0,     0],
        ...,
        [ 1749, 28756,  4126,  ...,     0,     0,     0],
        [  174, 30555, 30949,  ...,     0,     0,     0],
        [ 4904,  3205, 16570,  ...,     0,     0,     0]])
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [ ]:
train_validation_dataset=NerDataset(input_ids_train_validation,attention_masks_train_validation,dummy_label_train_validation)

predicted_labels_train_validation,_,_ = trainer_2.predict(train_validation_dataset)

#predicted_labels = np.argmax(predicted_labels_train_validation, axis=2)

formatted_predictions_train_validation = []

records_train_validation = listing_data.iloc[0:5000]["Record Number"].tolist()

In [ ]:
formatted_prediction_train_validation, word_labels_train_validation=get_formatted_prediction(records_train_validation,tokenizer_texts_train_validation,predicted_labels_train_validation,train_validation_data_tokens)

In [ ]:
#then we evaluate the f1 score on the whole train and alidation dataset with the results we get from predictiny by our model

sequence_train_validation=[]

for index in range(5000):
    # Filter the dataframe to get rows with the current index
    sentence_df = data[data['RN'] == index+1]

    # Get the tags, convert them to indices using tag2idx, and append to sequences
    tag_sequence = [tag2idx.get(tag, -1) for tag in sentence_df['Tag'].tolist()]
    sequence_train_validation.append(tag_sequence)

print(sequence_train_validation)

[[39, 35, 50, 51, 52, 39, 40, 40, 20, 62, 16, 43, 43, 4, 50, 51], [35, 36, 39, 0, 43, 0, 48, 43, 41, 42, 43, 48], [43, 35, 36, 55, 43, 16, 43], [35, 36, 39, 40, 40, 48, 20], [35, 50, 51, 39, 40, 43, 20, 39, 43, 20, 43, 20, 21, 43, 16, 17, 43, 62, 63, 43, 43, 43], [43, 35, 0, 55, 48, 66, 67, 39, 43, 39, 40, 20], [35, 36, 39, 20, 21], [35, 36, 39, 55, 48, 49, 49, 0, 43], [35, 50, 39, 40, 50, 45, 55, 48, 26, 20, 43, 43, 16, 43, 16, 43], [0, 55, 24, 48, 49, 64, 65, 48, 43, 48], [39, 40, 48, 55, 48, 0], [26, 55, 0, 39, 40, 48, 20, 43, 41, 48, 0], [35, 36, 39, 48, 52, 45, 48, 18, 26, 20, 43, 20], [35, 39, 40, 40, 0, 48, 64, 55, 45, 26, 43, 43, 16], [48, 49, 55, 35, 45, 43, 43, 16, 20, 43], [35, 55, 39, 40, 43, 20, 26, 45, 46, 0], [35, 39, 40, 0, 48, 55, 48, 26, 43, 20, 43, 43], [35, 50, 39, 0, 55, 20, 26, 6, 48, 49, 43], [50, 43, 41, 55, 43, 43, 16], [35, 50, 39, 52, 43, 0, 43, 20, 21, 43, 43, 16, 43, 16, 43, 43, 43, 26, 43, 55], [35, 50, 39, 40, 16, 17, 43, 43, 43, 43, 43, 43, 50, 51, 50, 3

In [ ]:
#run the report method to check the f1 score o whole words in the train and validation dataset using the method we generate the output file
from sklearn.metrics import classification_report

flattened_predictions_t_v = []
flattened_ground_truth_t_v = []
count=0
for pred, true in zip(word_labels_train_validation, sequence_train_validation):
    # Truncate predictions to the length of the ground truth for each sentence
    #print('length of prediction',len(pred))
    #print('length of true prediction',len(true))
    if len(flattened_predictions_t_v)!=len(flattened_ground_truth_t_v):
      break
    if len(pred)>len(true):
      truncated_pred = pred[:len(true)]

    #print('length of truncated prediction',len(truncated_pred))
      flattened_predictions_t_v.extend(truncated_pred)
      flattened_ground_truth_t_v.extend(true)
    elif len(pred)<len(true):
      truncated_ture=true[:len(pred)]

      flattened_predictions_t_v.extend(pred)
      flattened_ground_truth_t_v.extend(truncated_ture)

    else:
      flattened_predictions_t_v.extend(pred)
      flattened_ground_truth_t_v.extend(true)
    count+=1

# Compute weighted F1 score and classification report
print(len(flattened_ground_truth_t_v))
print(len(flattened_predictions_t_v))
report = classification_report(flattened_ground_truth_t_v, flattened_predictions_t_v, output_dict=True )

print(report)

55183
55183
{'0': {'precision': 0.8902474526928675, 'recall': 0.8692438885730529, 'f1-score': 0.879620307780814, 'support': 3518}, '1': {'precision': 0.9037037037037037, 'recall': 0.9172932330827067, 'f1-score': 0.9104477611940299, 'support': 266}, '2': {'precision': 0.8201058201058201, 'recall': 0.8031088082901554, 'f1-score': 0.8115183246073298, 'support': 579}, '3': {'precision': 0.8586956521739131, 'recall': 0.8681318681318682, 'f1-score': 0.8633879781420766, 'support': 91}, '4': {'precision': 0.8490566037735849, 'recall': 0.8256880733944955, 'f1-score': 0.8372093023255816, 'support': 109}, '5': {'precision': 0.8095238095238095, 'recall': 0.8947368421052632, 'f1-score': 0.8500000000000001, 'support': 19}, '6': {'precision': 0.8475409836065574, 'recall': 0.8167456556082149, 'f1-score': 0.831858407079646, 'support': 633}, '7': {'precision': 0.9591836734693877, 'recall': 0.9591836734693877, 'f1-score': 0.9591836734693877, 'support': 49}, '8': {'precision': 0.8305084745762712, 'recall'

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df_formatted_predictions_test = pd.DataFrame(formatted_prediction_train_validation, columns=['Record Number', 'Aspect Name', 'Aspect Value'])
csv_file_path = "/content/drive/MyDrive/Validation&Training/prediction_trai_validation_4000_425.csv"
df_formatted_predictions_test.to_csv(csv_file_path, sep="\t",index=False , encoding="utf-8")

In [ ]:
import pandas as pd
listing_data = pd.read_csv('/content/Listing_Titles.tsv', delimiter='\t', encoding='utf-8', error_bad_lines=False)
quiz_data_tokens = listing_data.iloc[5000:30000]["Title"].tolist()

tokenized_texts_test = [tokenizer.tokenize(t) for t in quiz_data_tokens]
input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                               maxlen=MAX_LEN, dtype="long", value=0.0,
                               truncating="post", padding="post")

attention_masks_test = [[float(i != 0.0) for i in ii] for ii in input_ids_test]

input_ids_test = torch.tensor(input_ids_test)
attention_masks_test = torch.tensor(attention_masks_test)

dummy_labels = torch.zeros_like(input_ids_test)

<ipython-input-42-dec9582d7312>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  listing_data = pd.read_csv('/content/Listing_Titles.tsv', delimiter='\t', encoding='utf-8', error_bad_lines=False)


In [ ]:
print(quiz_data_tokens)

In [ ]:
test_dataset = NerDataset(input_ids_test, attention_masks_test, dummy_labels)

predictions, _, _ = trainer_2.predict(test_dataset)
# = np.argmax(predictions, axis=2)

formatted_predictions = []

records = listing_data.iloc[5000:30000]["Record Number"].tolist()

In [ ]:
print(len(records))
print(len(tokenized_texts_test))
print(len(predicted_labels))

25000
25000
25000


In [ ]:
formatted_prediction_test,_=get_formatted_prediction(records,tokenized_texts_test,predictions,quiz_data_tokens)

In [ ]:
print(type(formatted_prediction_test))

<class 'list'>


In [ ]:
df_formatted_predictions_test = pd.DataFrame(formatted_prediction_test, columns=['Record Number', 'Aspect Name', 'Aspect Value'])
df_formatted_predictions_test = df_formatted_predictions_test.sort_values(by=['Record Number', 'Aspect Name'])
csv_file_path = "/content/drive/MyDrive/Validation&Training/prediction_test_4000_final.tsv"
df_formatted_predictions_test.to_csv(csv_file_path, sep="\t",index=False , encoding="utf-8",header=False)